In [ ]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit_mps_initializer.datatypes import QuantumState
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
from qiskit.quantum_info import partial_trace, DensityMatrix, state_fidelity
%matplotlib inline

psi = [1, 2, 3, 4, 5, 6, 7, 8]
phi = [1, 2, 3, 4, 5, 6, 7, 8]
n = 3  

delta = np.pi / 4  # Phase to be added
number_of_layers = 2

psi_state = QuantumState.from_dense_data(data=psi, normalize=True)
phi_state = QuantumState.from_dense_data(data=phi, normalize=True)

U_psi = psi_state.generate_mps_initializer_circuit(number_of_layers=number_of_layers)
U_phi = phi_state.generate_mps_initializer_circuit(number_of_layers=number_of_layers)

U_phi_dagger = U_phi.inverse()

psi_reg = QuantumRegister(n, name="ψ")
phi_reg = QuantumRegister(n, name="φ")
cregs = [ClassicalRegister(n, name='ancilla')]
qc = QuantumCircuit(psi_reg, phi_reg, *cregs)

#for psi
qc.append(U_psi, psi_reg)

#for phi
qc.append(U_phi, phi_reg)

def run_protocol(m):
    for k in range(1, m+1):
        for i in range(n):
            qc.cx(phi_reg[n-1-i], psi_reg[n-1-i], ctrl_state=0)

   # add phase
    qc.mcp(delta, psi_reg[0:-1], psi_reg[-1])

   # reset the circuit
    for i in range(n):
        qc.cx(phi_reg[i], psi_reg[i], ctrl_state=0)
    
    if k < m:
            qc.append(U_phi, phi_reg)        

    qc.save_statevector()

  #simulate
    sim = AerSimulator()
    result = sim.run(transpile(qc, sim)).result()

  # Full statevector
    full = result.get_statevector(qc)

    rho_psi = partial_trace(full, list(range(n, 2*n)))

 # Ideal target
    psi_vec = np.array(psi) / np.linalg.norm(psi)   # normalized ψ
    phi_vec = np.array(phi) / np.linalg.norm(phi)   # normalized φ
    phase_profile = np.exp(1j * delta * m * np.abs(phi_vec)**2)
    ideal_vec = psi_vec * phase_profile
    rho_ideal = DensityMatrix(ideal_vec)

    # fidelity
    fid = state_fidelity(rho_psi, rho_ideal)
    err = 1 - fid
    return fid, err


# --- run for different m ---
m_values = list(range(10, 101, 10))
fidelities, errors = [], []

for m in m_values:
    fid, err = run_protocol(m)
    fidelities.append(fid)
    errors.append(err)
    print(f"m={m}: Fidelity={fid:.6f}, Error={err:.6f}")

# --- plot results ---
plt.figure(figsize=(10,5))
plt.plot(m_values, fidelities, 'o-', label="Fidelity")
plt.plot(m_values, errors, 's-', label="Error")
plt.xlabel("Number of iterations (m)")
plt.ylabel("Value")
plt.title("Fidelity and Error vs Iterations")
plt.legend()
plt.grid(True)
plt.show()




# Measurement Z





qc.draw('mpl')


Current error: 1.3638079012812616
Current number of layers: 0
Current error: 1.3638079012812616
Current number of layers: 0
m=10: Fidelity=0.516995, Error=0.483005


QiskitError: 'Data for experiment "circuit-353" could not be found.'

In [ ]:
# --- enhanced visualization ---
fig, ax1 = plt.subplots(figsize=(10,6))

color1 = 'tab:blue'
ax1.set_xlabel("Number of iterations (m)")
ax1.set_ylabel("Fidelity", color=color1, fontsize=12)
ax1.plot(m_values, fidelities, 'o-', color=color1, label="Fidelity")
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_ylim(0, 1.05)

# Annotate the max fidelity point
max_fid_idx = np.argmax(fidelities)
ax1.annotate(f"Max Fidelity = {fidelities[max_fid_idx]:.3f}",
             xy=(m_values[max_fid_idx], fidelities[max_fid_idx]),
             xytext=(m_values[max_fid_idx], fidelities[max_fid_idx]+0.05),
             arrowprops=dict(arrowstyle="->", color=color1),
             color=color1)

# Create second y-axis for error
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel("Error", color=color2, fontsize=12)
ax2.plot(m_values, errors, 's--', color=color2, label="Error")
ax2.tick_params(axis='y', labelcolor=color2)

# Optionally, log scale for error (uncomment if errors are very small)
# ax2.set_yscale('log')

# Title and grid
plt.title("Fidelity and Error vs Iterations", fontsize=14, fontweight='bold')
fig.tight_layout()
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

# Combine legends from both axes
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="lower right")

plt.show()
